In [12]:
import pandas as pd
import datetime
import itertools
import requests
from bs4 import BeautifulSoup
import re
import os
import requests
import shutil
import urllib.request
import random

In [31]:
dir_url = "https://www.karriere.at/jobs?page=2"
hdr = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

# Request page content
dir_page_response = requests.get(dir_url, timeout=120, headers = hdr)

# Parse and turn into soup
dir_page_soup = BeautifulSoup(dir_page_response.content, 'html.parser')

# Returns max. number of pages to scrape
max_anzahl = dir_page_soup.find_all(class_ = 'm-pagination__meta')
max_anzahl = int(max_anzahl[0].contents[0].split()[2]) + 1

In [34]:
all_images = []

for i in range(1, max_anzahl):
    dir_url = 'https://www.karriere.at/jobs?page=' + str(i)
    dir_page_response = requests.get(dir_url, timeout=120, headers = hdr)
    dir_page_soup = BeautifulSoup(dir_page_response.content, 'html.parser')
    images = dir_page_soup.find_all(class_= 'm-jobsListItem__imageContainer')
    all_images.append(images)

# convert lists of list to single list
merged_images = list(itertools.chain(*all_images))

In [35]:
# get all the img-tags of the singe list
image_info = []
for a in merged_images:
    image_tag = a.findChildren("img")
    image_data = image_tag[0]['data-latesrc']
    image_info.append(image_data)

In [36]:
# make urls and remove all non-https tags
image_info = list(map( lambda x: x.replace('//', 'https://'), image_info))
image_info = [elem for elem in image_info if elem.startswith('https:')]

In [37]:
# loop over generated urls and save images
for url in image_info:
    random_number = round(random.random(), 5)
    url_name = url.split('/')    
    get_company_name = url_name[-1].split('.')[0:-1]
    ending = url_name[-1].split('.')[-1]
    get_company_name = " ".join(get_company_name)
    name_company = get_company_name + str(random_number) + str('.') + str(ending)
    urllib.request.urlretrieve(url, name_company)